### What for?

If you have two related operations which you’d like to execute as a pair,
with a block of code in between. Context managers allow you to do specifically that.

### Examples

File reading, these two are equivalent codes:

In [1]:
file = open('README.md', 'r')
try:
    content = file.read()
finally:
    file.close() # don't forget to close it!

print(len(content))

2211


In [2]:
with open('README.md', 'r') as file:
    content = file.read()

print(len(content))

2211


In [ ]:
# Django atomic transaction for DB

from django.db import transaction

with transaction.atomic():
    # This code executes inside a transaction.
    obj = MyModel.objects.first()
    obj.my_field = 13
    obj.save()


### Implementing context manager:

In [4]:
import time

class MyTimer:
    DEBUG = False
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        if self.DEBUG:
            print(f'__enter__ ({self.code_name})')
        self.start_time = time.time()

    def __exit__(self, exception_type, exception_value, traceback):
        if self.DEBUG:
            print(f'__exit__ ({self.code_name})')
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')


a = 10
b = 100

with MyTimer("code1"):
    time.sleep(1)
    c = a ** b

Code: "code1" took 1.0036


In [5]:
with MyTimer("code2"):
    time.sleep(1)
    for i in range(a):
        with MyTimer(f"For loop [{i}]"):
            time.sleep(0.2)
            c = a ** a

Code: "For loop [0]" took 0.2038
Code: "For loop [1]" took 0.2001
Code: "For loop [2]" took 0.2022
Code: "For loop [3]" took 0.2038
Code: "For loop [4]" took 0.2050
Code: "For loop [5]" took 0.2046
Code: "For loop [6]" took 0.2041
Code: "For loop [7]" took 0.2045
Code: "For loop [8]" took 0.2052
Code: "For loop [9]" took 0.2041
Code: "code2" took 3.0446


In [6]:
my_timer = MyTimer("code3")
my_timer.DEBUG = True

with my_timer:
    time.sleep(1)

__enter__ (code3)
__exit__ (code3)
Code: "code3" took 1.0032


`__enter__`  can return the object that will be used in `with <manager> as <return_from_enter>:`

In [7]:
class MyTimer2:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return 3

    def __exit__(self, exception_type, exception_value, traceback):
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')

my_timer2 = MyTimer2('code4')
with my_timer2 as something:
    print(something)  # something is `3`
    time.sleep(1)


3
Code: "code4" took 1.0033


In [8]:
class MyTimer3:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None
        self.betweens = []

    def __enter__(self):
        self.start_time = time.time()
        return self

    def between(self):
        self.betweens.append(time.time() - self.start_time)

    def __exit__(self, exception_type, exception_value, traceback):
        betweens = ', '.join(f'{x:.3f}' for x in self.betweens)
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}. Betweens: {betweens}')


with MyTimer3('code5') as timer:
    print(timer)  # time is MyTimer3 object
    time.sleep(1)
    timer.between()
    time.sleep(0.2)
    timer.between()
    time.sleep(0.2)


Code: "code5" took 1.4131. Betweens: 1.005, 1.209


In [9]:
class MyTimer4:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        print("Will __exit__ be executed?")
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')


with MyTimer4('code6') as timer:
    time.sleep(0.2)
    raise Exception("Ooops")
    print('Will it reach this code?')

print('outside of with')

Will __exit__ be executed?
Code: "code6" took 0.2040


Exception: Ooops

`__exit__` should return boolean, whether exception is handled by it context manager or not

In [10]:
class MyTimer5:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        print("Will __exit__ be executed?")
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')
        print(exception_type, exception_value)
        if isinstance(exception_value, KeyError):
            return True

In [11]:
with MyTimer5('code6') as timer:
    time.sleep(0.2)
    raise KeyError("Ooops")
    print('Will it reach this code?')

print('outside of with')

Will __exit__ be executed?
Code: "code6" took 0.2041
<class 'KeyError'> 'Ooops'
outside of with


In [12]:
with MyTimer5('code6') as timer:
    time.sleep(0.2)
    raise Exception("Ooops")
    print('Will it reach this code?')

print('outside of with')


Will __exit__ be executed?
Code: "code6" took 0.2048
<class 'Exception'> Ooops


Exception: Ooops

### Implementing context manager as function

Classic example

In [13]:
from contextlib import contextmanager

@contextmanager
def my_timer(filename):
    f = open(filename, 'r')
    try:
        yield f # you can yield nothing with `yield` if you yield something it can be used with "as ..."
    finally:
        f.close()


with my_timer('README.md') as file:
    print(f'{file=}')
    content = file.read()
    print(len(content))

file=<_io.TextIOWrapper name='README.md' mode='r' encoding='UTF-8'>
2211


Timer example

In [14]:
from contextlib import contextmanager

@contextmanager
def my_timer(code_name):
    start_time = time.time()
    try:
        yield
    finally:
        print(f'Code: "{code_name}" took {time.time() - start_time:.4f}')

with my_timer('code 7'):
    time.sleep(1)

Code: "code 7" took 1.0042
